In [1]:
import streamlit as st
import pandas as pd
import pydeck as pdk
import hashlib
import glob
import streamlit.components.v1 as components
import plotly.graph_objects as go
import matplotlib.colors as mcolors

In [2]:
# 📂 Ruta donde están los archivos
ruta_archivos = "reportes/*.xlsx"

# 🧾 Lista de todos los archivos que coincidan
archivos = glob.glob(ruta_archivos)

# 📊 Leer y concatenar todos los archivos
df_mrk = pd.concat(
    [pd.read_excel(archivo, sheet_name="Resumen ruta") for archivo in archivos],
    ignore_index=True
)

In [3]:
from datetime import time

# 🕒 1. Detectar columnas con "(hr)"
cols_hr = [col for col in df_mrk.columns if '(hr)' in col]
print("Columnas a convertir:", cols_hr)

# 🧭 2. Normalizar y convertir
for col in cols_hr:
    # Si algún valor es datetime.time, conviértelo a string "HH:MM:SS"
    df_mrk[col] = df_mrk[col].apply(
        lambda x: x.strftime('%H:%M:%S') if isinstance(x, time) else x
    )
    # Luego convertir a timedelta
    df_mrk[col] = pd.to_timedelta(df_mrk[col], errors='coerce')

# 📊 Verificar
df_mrk.info()

Columnas a convertir: ['Horario de sincronización (hr)', 'Inicio de día (hr)', 'Primer cliente (hr)', 'Tiempo cliente-transmisión (hr)', 'Tiempo de mercado (hr)', 'Tiempo de servicio (hr)', 'Traslado clientes(hr)', 'Transmisión de pedido (hr)', 'Tiempo de jornada (hr)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71061 entries, 0 to 71060
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype          
---  ------                           --------------  -----          
 0   Zona                             71061 non-null  object         
 1   Región                           71061 non-null  object         
 2   Gerencia                         71061 non-null  object         
 3   Jefatura                         71061 non-null  object         
 4   Ruta                             71061 non-null  object         
 5   Fecha inicio                     71061 non-null  datetime64[ns] 
 6   Fecha fin                        71061 non-null  dateti

In [4]:
df_mrk.head()

,Zona,Región,Gerencia,Jefatura,Ruta,Fecha inicio,Fecha fin,Geoefectividad,Efectividad omnicanal,Geoeficiencia,...,Tiempo de mercado (hr),Tiempo de servicio (%),Tiempo de servicio (hr),Traslado clientes (%),Traslado clientes(hr),Transmisión de pedido (%),Transmisión de pedido (hr),Tiempo de jornada (%),Tiempo de jornada (hr),Usabilidad
0,BAJIO,CELAYA,AC01,AC2,AC0067,2025-10-02,2025-10-02,0.607,0.732,0.929,...,0 days 10:18:49,0.232,0 days 03:46:36,0.964,0 days 06:08:25,1,0 days 18:30:28,0,0 days 11:22:25,0.581
1,BAJIO,CELAYA,AC01,AC2,AC0075,2025-10-02,2025-10-02,0.542,0.762,0.983,...,0 days 09:30:48,0.033,0 days 01:41:07,0.844,0 days 07:33:34,1,0 days 18:19:46,0,0 days 11:13:40,0.319
2,BAJIO,CELAYA,AC01,AC2,AC0076,2025-10-02,2025-10-02,0.763,0.864,1.000,...,0 days 09:53:48,0.101,0 days 03:06:39,0.983,0 days 06:21:02,1,0 days 18:40:57,1,0 days 10:20:00,0.541
3,BAJIO,CELAYA,AC01,AC2,AC0077,2025-10-02,2025-10-02,0.704,0.925,0.926,...,0 days 10:53:55,0.222,0 days 03:22:05,0.833,0 days 06:10:57,0,0 days 19:14:55,0,0 days 11:31:05,0.357
4,BAJIO,CELAYA,AC01,AC2,AC0078,2025-10-02,2025-10-02,0.679,0.857,0.964,...,0 days 09:56:34,0.232,0 days 03:51:48,0.982,0 days 05:13:27,1,0 days 17:48:30,1,0 days 10:23:51,0.680


In [5]:
df_catalogo = pd.read_csv('catalogo.csv', encoding='utf-8')
df_catalogo = df_catalogo[['Ruta','Tipo','Descripción Tipo','TPV','Región Comercial_Act 2026']]

df_mrk_full = df_mrk.merge(how='left', right=df_catalogo, left_on='Ruta', right_on='Ruta')
df_mrk_full.head()

,Zona,Región,Gerencia,Jefatura,Ruta,Fecha inicio,Fecha fin,Geoefectividad,Efectividad omnicanal,Geoeficiencia,...,Traslado clientes(hr),Transmisión de pedido (%),Transmisión de pedido (hr),Tiempo de jornada (%),Tiempo de jornada (hr),Usabilidad,Tipo,Descripción Tipo,TPV,Región Comercial_Act 2026
0,BAJIO,CELAYA,AC01,AC2,AC0067,2025-10-02,2025-10-02,0.607,0.732,0.929,...,0 days 06:08:25,1,0 days 18:30:28,0,0 days 11:22:25,0.581,PR,Preventa Comercial,ZPV,Celaya
1,BAJIO,CELAYA,AC01,AC2,AC0075,2025-10-02,2025-10-02,0.542,0.762,0.983,...,0 days 07:33:34,1,0 days 18:19:46,0,0 days 11:13:40,0.319,PR,Preventa Comercial,ZPV,Celaya
2,BAJIO,CELAYA,AC01,AC2,AC0076,2025-10-02,2025-10-02,0.763,0.864,1.000,...,0 days 06:21:02,1,0 days 18:40:57,1,0 days 10:20:00,0.541,PR,Preventa Comercial,ZPV,Celaya
3,BAJIO,CELAYA,AC01,AC2,AC0077,2025-10-02,2025-10-02,0.704,0.925,0.926,...,0 days 06:10:57,0,0 days 19:14:55,0,0 days 11:31:05,0.357,PR,Preventa Comercial,ZPV,Celaya
4,BAJIO,CELAYA,AC01,AC2,AC0078,2025-10-02,2025-10-02,0.679,0.857,0.964,...,0 days 05:13:27,1,0 days 17:48:30,1,0 days 10:23:51,0.680,PR,Preventa Comercial,ZPV,Celaya


In [6]:
df_mrk_cp = df_mrk_full[df_mrk_full['Zona'] == 'CENTRO-PACIFICO']
df_mrk_cp = df_mrk_cp[df_mrk_cp['Tipo'].isin(['PR', 'PCOP', 'FC', 'MA', 'PRTDC'])]

df_mrk_cp['Región'] = df_mrk_cp['Región'].where(
~df_mrk_cp['Tipo'].isin(['MA', 'PRTDC'])
).fillna(df_mrk_cp['Tipo'].replace({'MA': 'MAYORISTAS', 'PRTDC': 'MODERNO'}))
df_mrk_cp['Semana'] = df_mrk_cp['Fecha inicio'].dt.isocalendar().week
df_mrk_cp.rename(columns={'Región Comercial_Act 2026': 'Nombre UO'}, inplace=True)


#df_mrk_cp_pivot = df_mrk_cp.pivot_table(index=['Semana','Fecha inicio'],columns='Región',values=['Geoeficiencia'], aggfunc='mean')

df_mrk_cp
df_mrk_cp.to_parquet('bdd_mrk_cp.parquet')